In [1]:
using Pkg
Pkg.activate(".")

using Revise, PtyLab, TestImages, ImageShow, IndexFunArrays, FFTW, HDF5, Noise, FourierTools, CUDA
using EllipsisNotation
FFTW.set_num_threads(4)

  Activating environment at `~/PtyLab.jl/examples/Project.toml`
┌ Info: Precompiling PtyLab [81acdcc5-82ff-4242-81ec-b83d268fd5e7]
└ @ Base loading.jl:1342


## Object

In [2]:
img_abs = Float32.(testimage("fabio_gray_512"))[200:400, 200:400]
img_phase = Float32.(testimage("resolution_test_512"))[200:400, 200:400]
object = img_abs .* cispi.(2 .* img_phase)

complex_show(object);

## Random Grid

In [3]:
grid_size = size(object)
tile_size = (100, 100)

grr = PtyLab.grid_regular_rand(grid_size, tile_size, (15, 15), 30);
@show grr.overlap
show_grid(grr, only_points=false);

grr.overlap = (0.9299999999999999, 0.9299999999999999)


## Probes

In [4]:
probe1 = IndexFunArrays.gaussian(Float32, tile_size, offset=(45, 45), scale=(0.020, 0.10)) .* cis.(Float32(2π) .* 
     4 .* gaussian(Float32, tile_size, scale=(0.003, 0.030)));


probe2 = IndexFunArrays.gaussian(Float32, tile_size, offset=(50, 50), scale=(0.040, 0.020)) .* cis.(Float32(2π) .* 
     4 .* gaussian(Float32, tile_size, scale=(0.001, 0.010)));



complex_show([probe1 probe2]);

## Simulate and Store dataset

In [5]:
ptychogram = zeros(Float32, (tile_size..., length(grr.tiles)));
p = Params()
o2d, d2o = Fraunhofer(probe1, fftshiftFlag=true);

for (i, t) in enumerate(grr.tiles)
    ptychogram[:, :, i] = abs2.(   o2d(view(object, t.i₁:t.i₂,  t.j₁:t.j₂) .* probe1)
                                .+ o2d(view(object, t.i₁:t.i₂,  t.j₁:t.j₂) .* probe2)
                                )
    
end


lambda = 633f-9
z = 50f-3
dxd = 10f-6
Nd = size(ptychogram, 1)
dxo = lambda * z / (Nd * dxd)

fid_new = h5open("simulated_ptychography.hdf5", "w");
fid_new["Nd"] = Nd
fid_new["No"] = size(img_abs, 1)
fid_new["dxd"] = 10f-6
fid_new["encoder"] = PtyLab.encoder(grr, dxo, offset=(50, 50))
fid_new["wavelength"] = lambda
fid_new["entrancePupilDiameter"] = dxo * 30
fid_new["zo"] = z
fid_new["ptychogram"] = ptychogram
close(fid_new)
#@view_image ptychogram;

## Load Dataset

In [6]:
experimentalData = ExperimentalDataCPM("simulated_ptychography.hdf5");

reconstruction = ReconstructionCPM(experimentalData, cuda=false);
reconstruction = PtyLab.initializeObjectProbe!(reconstruction);

engine = PtyLab.ePIE()
params2 = Params()

┌ Info: Reading HDF5.File: (read-only) simulated_ptychography.hdf5 was successful
└ @ PtyLab /home/fxw/PtyLab.jl/src/ExperimentalData.jl:55
┌ Warning: Currently No = 4 * Np, just as guess, should be improved
└ @ PtyLab /home/fxw/PtyLab.jl/src/utils_calc.jl:91


Params
  fftshiftFlag: Bool false
  transposePtychogram: Bool true
  intensityConstraint: PtyLab.IntensityConstraintStandard <: PtyLab.IntensityConstraint
  propagatorType: Fraunhofer (function of type typeof(Fraunhofer))
  randPositionOrder: Bool true
  comStabilizationSwitch: Bool true


## Reconstruction



In [39]:
reconstruction = ReconstructionCPM(experimentalData, cuda=false);
#reconstruction = PtyLab.initializeObjectProbe!(reconstruction);

#engines.
params2.fftshiftFlag = false
engine.betaProbe = 0.75f0
engine.betaObject = 0.75f0

params2.transposePtychogram = false
params2.comStabilizationSwitch = true
reconstruction.npsm = 2
reconstruction = PtyLab.initializeObjectProbe!(reconstruction);
engine.numIterations = 50

CUDA.@time p, o = PtyLab.reconstruct(engine, params2, reconstruction);

┌ Warning: Currently No = 4 * Np, just as guess, should be improved
└ @ PtyLab /home/fxw/PtyLab.jl/src/utils_calc.jl:91
┌ Warning: gimmel is currently estimated as `100 * eps(Float32)`
└ @ PtyLab /home/fxw/PtyLab.jl/src/Engines.jl:56
Progress: 100%|█████████████████████████████████████████| Time: 0:00:11


 11.642305 seconds (3.24 M CPU allocations: 207.804 MiB, 0.28% gc time)


In [41]:
complex_show(Array(reconstruction.object[:, :, 1,1,1,1]));

In [23]:
complex_show(Array(reconstruction.probe[:, :, 1,1,2,1]));

In [21]:
complex_show(Array(reconstruction.probe[:, :, 1,1,1,1]));

In [20]:
complex_show(probe1[:, :, 1,1,1,1]);

In [19]:
complex_show(probe2[:, :, 1,1,1,1]);

In [18]:
complex_show(reconstruction.probe[:, :, 1,1,1,1] .+ reconstruction.probe[:, :, 1,1,2,1]);

In [17]:
complex_show((probe2[:, :, 1,1,1,1] .+ probe1[:, :,1,1,1,1]));